### Imports

In [1]:
import ee
import geemap
import math
ee.Initialize()

### Datasets

In [2]:
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
admin = ee.FeatureCollection("FAO/GAUL/2015/level0")
active_mines = ee.FeatureCollection("users/raymondeah/cod_mines_curated_all_opendata_p_ipis")

### Regions

In [3]:
roi1 = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

roi2 = ee.Geometry.Polygon(
        [[[30.246670960050185, 1.7911944738716732],
          [30.246670960050185, 1.7103797163160706],
          [30.356362579923232, 1.7103797163160706],
          [30.356362579923232, 1.7911944738716732]]])

all = ee.Geometry.Polygon(
        [[[22.92587933089792, 5.5911393992628495],
          [22.92587933089792, -13.776973382582892],
          [31.45126995589792, -13.776973382582892],
          [31.45126995589792, 5.5911393992628495]]])

rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])

emily_area = ee.Geometry.Polygon(
        [[[25.599767416235804, -10.585427828449394],
          [25.599767416235804, -10.82359555219894],
          [25.977297323462366, -10.82359555219894],
          [25.977297323462366, -10.585427828449394]]])

emily_area2 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]])

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

### Visualization Parameters

In [4]:
rgb_vis_ls5 = {
    'min': 8211.7,
    'max': 11077.3,
    'bands': ['SR_B3', 'SR_B2', 'SR_B1']
}

rgb_vis_s2 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

rgb_vis_s2_scaled = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2']
}

vvVis = {
    'opacity': 1,
    'bands': ["VV"],
    'min': -13.375350094403828,
    'max': -3.6299557970059952,
    'gamma': 1
}

vhVis = {
    'opacity': 1,
    'bands': ["VH"],
    'min': -21.945806268850372,
    'max': -10.8330283399149,
    'gamma': 1
}


### Classification Training Data

In [5]:
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.566649352977876, 3.131936594576548]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.594069479589553, 3.143136886698536]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.578229613742543, 3.109209943820683]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.581158585986806, 3.1067030888196494]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.55884031463487, 3.11517956941471]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.581819366025293, 3.132088091449441]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.595938514279688, 3.131445322675145]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.59962923388418, 3.1347448648629315]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.565039466428125, 3.104877202480009]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.607741346488673, 3.1193612914096605]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

In [6]:
bare_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.588761955782978, 3.110552597531067]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.590113789126484, 3.1101669284095776]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.570400011069342, 3.114240974803364]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.56574369621949, 3.114048140935224]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.566300979571633, 3.1296952794920045]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.61279975505381, 3.134623171579494]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.620696178393654, 3.1514206812868597]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.60730659099131, 3.15656272195781]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.609366527514748, 3.1041127202608414]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.561301341967873, 3.0982847807562592]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

### Cloud Masking Functions

In [7]:
# cloud mask landsat
def maskL457sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    return image.addBands(opticalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)

In [8]:
# cloud mask sentinel
def maskS2clouds(image):
    qa = image.select('QA60')

    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).select("B.*") \
        .copyProperties(image, ["system:time_start"])

### Classification/Areal Change

In [9]:
training = bare.merge(vegetation)

composite1985 = ls5 \
        .filter(ee.Filter.bounds(roi1)) \
        .filter(ee.Filter.date('1985-01-01', '1986-01-01')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .map(maskL457sr) \
        .select('SR_B.*') \
        .median() \
        .clip(roi1)

# Overlay the point on the image to get training data.
training = composite1985.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_ls = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite1985.bandNames()
})

# Classify the image.
classified = composite1985.classify(classifier_ls)

# Map.addLayer(classified,
#   {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, '1985 LS5 Classified')
# Map

In [10]:
training = bare_s.merge(vegetation_s)

composite2021 = s2 \
        .filter(ee.Filter.bounds(roi1)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median() \
        .clip(roi1) \

# Overlay the point on the image to get training data.
training = composite2021.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_s2 = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite2021.bandNames()
})

# Classify the image.
classified = composite2021.classify(classifier_s2)

# Map.addLayer(classified,
#   {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, '2021 S2 Classified')
# Map

### Index and Band Calculation Functions

In [11]:
def calculate_percentage_change(feature):
    g = feature.geometry()
    
    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .map(maskL457sr) \
        .select('SR_B.*') \
        .median() \
        .clip(g)

    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median() \
        .clip(g)
    
    left_classified = composite_ls.classify(classifier_ls)
    right_classified = composite_s2.classify(classifier_s2)
    
    # Reclassify from 0-1 to 1-2
    left_classes = left_classified.remap([0, 1], [1, 2])
    right_classes = right_classified.remap([0, 1], [1, 2])

    initial_vegetation = left_classes.eq(2)
    bare = right_classes.eq(1)
    vegetation_to_bare = initial_vegetation.And(bare)
    
    area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())

    area = area_image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

    area_initial_vegetation = left_classified.eq(1).multiply(ee.Image.pixelArea())

    area2 = area_initial_vegetation.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)
    
    percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)
    
    ## account for loss of vegetation from before 1985
    
    #area of image
    total_area = g.area()
    total_SqKm = ee.Number(total_area).divide(1e6)
    #area of bare earth 2021
    bare_image = bare.multiply(ee.Image.pixelArea())

    area3 = bare_image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_bare = ee.Number(area3.get('remapped')).divide(1e6)
    
    percent_bare = area_bare.divide(total_SqKm).multiply(100)
        
    return feature.set('percent loss',  percent_loss).set('percent bare', percent_bare)

In [12]:
def calculate_sar_vh(feature):
    g = feature.geometry()
    
    # Images and Bands
    filtered = s1 \
        .filter(ee.Filter.eq('instrumentMode','IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filter(ee.Filter.eq('resolution_meters',10)) \
        .filter(ee.Filter.intersects('.geo', g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .select('VH')

    composite = filtered.median().clip(g)
    
    #area of image
    total_area = g.area()
    total_SqKm = ee.Number(total_area).divide(1e6)
    
    #area of possible mines
    area_mines = composite.lt(-19).rename('mines')
    connect = area_mines.connectedPixelCount(25);
    area_mines = area_mines.updateMask(connect.gt(8));
    area_mines = area_mines.multiply(ee.Image.pixelArea())

    area = area_mines.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 30,
      'maxPixels': 1e10
    })

    mines_SqKm = ee.Number(area.get('mines')).divide(1e6)

    percent_mine = ee.Number(mines_SqKm.divide(total_SqKm).multiply(100))
    
    return feature.set('vh_percent', percent_mine)

In [13]:
def calculate_nir_g(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median() \
        .clip(g)
    nir_g_s2 = composite_s2.normalizedDifference(['B8', 'B3']).rename('NIR/G')

    # Average NIR/G   
    stats_s2 = nir_g_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': nir_g_s2.geometry(),
        'scale': 30
    })
    
    avg_s2 = ee.Number(stats_s2.get('NIR/G'))
    
    return feature.set('nir/g',  avg_s2)

In [14]:
def calculate_swirb(feature):
    g = feature.geometry()
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median() \
        .clip(g) 
    swirb = composite_s2.normalizedDifference(['B11', 'B2']).rename('SWIR1/B')
    stats = swirb.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': g,
        'scale': 30,
        'maxPixels': 1e10
    })
    swirb_val = stats.get('SWIR1/B')
    
    return feature.set('swir/b', swirb_val)

In [15]:
def calculate_ndmi_change(feature):
    g = feature.geometry()
    
    # Images and Bands
    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .map(maskL457sr) \
        .select('SR_B.*') \
        .median() \
        .clip(g)
    ndmi_ls = composite_ls.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDMI')
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median() \
        .clip(g)
    ndmi_s2 = composite_s2.normalizedDifference(['B8', 'B11']).rename('NDMI')

    # Average NDMI
    stats_ls = ndmi_ls.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi_ls.geometry(),
        'scale': 30
    })    
    stats_s2 = ndmi_s2.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi_s2.geometry(),
        'scale': 30
    })
    
    avg_ls = ee.Number(stats_ls.get('NDMI'))
    avg_s2 = ee.Number(stats_s2.get('NDMI'))
    
    # NDMI Loss
    ndmi_loss = avg_ls.subtract(avg_s2)
    
    return feature.set('ndmi loss',  ndmi_loss)

### Squares

In [16]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares

edit: remove some stuff from geometry produced
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]])
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

### Revised Workflow

##### Square -> Vegetation loss -> SAR value -> SWIR1/BLUE value -> NIR/G value -> Final Product

##### Filtering Levels

In [17]:
def filter_by_vegetation_loss(squares, threshold1, threshold2):
    with_percent_change = squares.map(calculate_percentage_change)
    passed = with_percent_change.filter((ee.Filter.gt('percent loss', threshold1)).Or(ee.Filter.gt('percent bare', threshold2)))
    return passed

In [18]:
def filter_by_vh_percent(squares, threshold):
    with_change = squares.map(calculate_sar_vh)
    passed = with_change.filter(ee.Filter.gt('vh_percent', threshold))
    return passed

In [19]:
def filter_by_nir_g(squares, threshold):
    with_change = squares.map(calculate_nir_g)
    passed = with_change.filter(ee.Filter.lte('nir/g', threshold))
    return passed

In [20]:
def filter_by_swir1_b(squares, threshold):
    with_swir1_b = squares.map(calculate_swirb)
    passed = with_swir1_b.filter(ee.Filter.lte('swir/b', threshold))
    return passed

In [21]:
def filter_by_ndmi_loss(squares, threshold):
    with_change = squares.map(calculate_ndmi_change)
    passed = with_change.filter(ee.Filter.gt('ndmi loss', threshold))
    return passed

##### Creating Images for Visualization

In [22]:
def create_whole_composite(geometry):
    composite = s2 \
        .filter(ee.Filter.bounds(geometry)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select('B.*') \
        .median().clip(geometry)
    return composite

##### Running the Code on Each Chosen Region

In [23]:
def applyRoutine(geometry, zoom, square_size):
    Map = geemap.Map()
    Map.centerObject(geometry, zoom)
    
    segments = ee.FeatureCollection(create_segments(geometry, square_size)).filter(ee.Filter.bounds(drc))
    
    passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 20)
    passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
    passed_swir_b = filter_by_swir1_b(passed_sar_vh, 0.62)
    passed_nir_g = filter_by_nir_g(passed_swir_b, 0.45)
    
    composites = create_whole_composite(geometry)
    
    Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
    Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
    Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', opacity=0.7)
    Map.addLayer(passed_swir_b, {'color': 'cyan'}, 'Passed SWIR1/B', opacity=0.7)
    Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
    Map.addLayer(active_mines, {'color': 'black'}, 'mapbox active mines')
    
    return Map

In [24]:
# """
# 1. 10x10km squares (~15000)
# 2. applyRoutine(1km) for each 10x10km square
# 3. combine into 1 collection
# 4. export
# """

# Map = geemap.Map()
# Map.centerObject(all, 5)

# s = create_segments(all, 10)[5000:5005]
# smallest = []
# for square in s:
#     bruh = create_segments(square, 1)
#     smallest += bruh
# segments = ee.FeatureCollection(s[5000:5010]).filter(ee.Filter.bounds(drc))
# Map.addLayer(all, {'color': 'blue'}, 'bounding box')
# Map.addLayer(segments, {'color': 'cyan'}, '10x10km')
# Map.addLayer(ee.FeatureCollection(smallest), {'color': 'purple'}, '1x1km')
# Map

Map(center=[-4.064367300737955, 27.18857464339792], controls=(WidgetControl(options=['position', 'transparent_…

In [34]:
Map = applyRoutine(roi1, 13, 1)
Map

Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…

In [26]:
# Map = applyRoutine(rishiArea, 13, 0.5)
# Map

In [27]:
# Map = applyRoutine(emily_area, 11, 1)
# Map

In [28]:
# Map = geemap.Map()
# Map.centerObject(emily_area2, 11)

# segments = ee.FeatureCollection(create_segments(emily_area2, 0.5)).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 10)
# passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
# passed_nir_g = filter_by_nir_g(passed_sar_vh, 0.45)
# # passed_ndmi_loss = filter_by_ndmi_loss(passed_nir_g, 0.03)

# composites = create_whole_composite(emily_area2)

# Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
# Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
# Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
# # Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', opacity=0.7)
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

# Map

In [29]:
# geometry = ee.Geometry.Polygon(
#         [[[28.6, -1.5],
#           [28.6, -2],
#           [29, -2],
#           [29, -1.5]]])

# Map = geemap.Map()
# Map.centerObject(geometry, 11)

# segments = ee.FeatureCollection(create_segments(geometry, 0.5)).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 10)
# passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
# passed_nir_g = filter_by_nir_g(passed_sar_vh, 0.45)
# # passed_ndmi_loss = filter_by_ndmi_loss(passed_nir_g, 0.03)

# composites = create_whole_composite(geometry)

# Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
# Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
# Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
# # Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', opacity=0.7)
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

# Map

In [30]:
# geometry2 = ee.Geometry.Polygon(
#         [[[28, -1.95],
#           [28, -2.25],
#           [28.5, -2.25],
#           [28.5, -1.95]]])

# Map = geemap.Map()
# Map.centerObject(geometry2, 11)

# segments = ee.FeatureCollection(create_segments(geometry2, 0.5)).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 10)
# passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
# passed_nir_g = filter_by_nir_g(passed_sar_vh, 0.45)
# # passed_ndmi_loss = filter_by_ndmi_loss(passed_nir_g, 0.03)

# composites = create_whole_composite(geometry2)

# Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
# Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
# Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
# # Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', opacity=0.7)
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

# Map

In [31]:
# geometry3 = ee.Geometry.Polygon(
#         [[[27.380443620904487, -2.13211708933348],
#           [27.380443620904487, -2.3846061694222813],
#           [27.786937761529487, -2.3846061694222813],
#           [27.786937761529487, -2.13211708933348]]])

# Map = geemap.Map()
# Map.centerObject(geometry3, 11)

# segments = ee.FeatureCollection(create_segments(geometry3, 0.5)).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 10)
# passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
# passed_nir_g = filter_by_nir_g(passed_sar_vh, 0.45)
# # passed_ndmi_loss = filter_by_ndmi_loss(passed_nir_g, 0.03)

# composites = create_whole_composite(geometry3)

# Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
# Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
# Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
# # Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', opacity=0.7)
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

# Map

In [32]:
# geometry4 = ee.Geometry.Polygon(
#         [[[25.73867143063492, -9.314354178894073],
#           [25.73867143063492, -9.606950141895844],
#           [26.26052201657242, -9.606950141895844],
#           [26.26052201657242, -9.314354178894073]]])

# Map = geemap.Map()
# Map.centerObject(geometry4, 11)

# segments = ee.FeatureCollection(create_segments(geometry4, 0.5)).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss = filter_by_vegetation_loss(segments, 20, 10)
# passed_sar_vh = filter_by_vh_percent(passed_vegetation_loss, 5)
# passed_nir_g = filter_by_nir_g(passed_sar_vh, 0.45)
# # passed_ndmi_loss = filter_by_ndmi_loss(passed_nir_g, 0.03)

# composites = create_whole_composite(geometry4)

# Map.addLayer(composites, rgb_vis_s2_scaled, 'Median Composites 2021')
# Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', False, opacity=0.7)
# Map.addLayer(passed_sar_vh, {'color': 'purple'}, 'Passed SAR VH', False, opacity=0.7)
# Map.addLayer(passed_nir_g, {'color': 'blue'}, 'Passed NIR/G', opacity=0.7)
# # Map.addLayer(passed_ndmi_loss, {'color': 'orange'}, 'Passed NDMI Loss', opacity=0.7)
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

# Map

In [33]:
segments = create_segments(all, 10)
print(len(segments))

19995
